# Initial Setup

## Install Weaviate Python Client v4
> This notebook was created with Weaviate `1.25` and the Weaviate Client `4.6`

Run the below command to install the latest version of the Weaviate Python Client v4.

In [ ]:
!pip show weaviate-client

## Deploy Weaviate

Weaviate offers 3 deployment options:
* Embedded
* Self-hosted - with Docker Compose
* Cloud deployment - [Weaviate Cloud Service](https://console.weaviate.cloud/)

# Time to Build

## Connect to Weaviate

* If you are new to OpenAI, register at [https://platform.openai.com](https://platform.openai.com/) and head to [https://platform.openai.com/api-keys](https://platform.openai.com/api-keys) to create your API key.
* If you are new to Cohere, register at [https://cohere.com](https://https://cohere.com) and head to [https://dashboard.cohere.com/api-keys](https://dashboard.cohere.com/api-keys) to create your API key.

In [ ]:
import weaviate, json

# Connect to the local instance
client = weaviate.connect_to_local()

client.is_ready()

## Create a collection
[Weaviate Docs - collection creation and configuration](https://weaviate.io/developers/weaviate/configuration/schema-configuration)

In [ ]:
# TODO: Create a collection called Jeopardy - with a vectorizer

## Import data

### Sample Data

In [ ]:
import json

data_10 = json.load(open("./jeopardy_tiny.json"))

print(json.dumps(data_10, indent=2))

### Insert Many
[Weaviate Docs - insert many](https://weaviate.io/developers/weaviate/manage-data/import)

In [ ]:
# TODO: Insert data

### Data preview

In [ ]:
# TODO: Show data preview

In [ ]:
# TODO: Show data preview - with vectors

### A super quick query example

In [ ]:
questions = client.collections.get("Questions")
response = questions.query.near_text(
    query="African animals",
    # query="weather",
    limit=2
)

for item in response.objects:
    print(item.properties)

## Create a collection with a Generative module

In [ ]:
# TODO: Create a new collection with a vectorizer and generative model

### Import data - 1k objects

In [ ]:
data_1k = json.load(open("./jeopardy_1k.json"))

print(json.dumps(data_1k, indent=2))

In [ ]:
# Insert data
questions = client.collections.get("Questions")

# TODO: use batch import here

# questions.data.insert_many(data_1k)